In [1]:
import numpy as np
import math
import time
import logging
from data.data_loader import Dataset
from misc.preferences import PREFERENCES
from misc.run_configuration import get_default_params, OutputLayerType, LearningSchedulerType, OptimizerType, hyperOpt_goodParams, default_params
from misc import utils

from optimizer import get_optimizer
from criterion import NllLoss, LossCombiner

from models.transformer.encoder import TransformerEncoder
from models.jointAspectTagger import JointAspectTagger
from trainer.train import Trainer
import pprint
from data.germeval2017 import germeval2017_dataset as dsl

In [ ]:
PREFERENCES.defaults(
	data_root='./data/data/amazon/splits',
	data_train='train.pkl',    
	data_validation='val.pkl',
	data_test='test.pkl',
	source_index=0,
	target_vocab_index=1,
	file_format='pkl'
)
from data.amazon import amazon_dataset as dsl

# Loading Functions

These functions will load the dataset and the model. The run configuration will determine the architecture and hyper parameters.

In [2]:
def load_model(dataset, rc, experiment_name):
    loss = LossCombiner(4, dataset.class_weights, NllLoss)
    transformer = TransformerEncoder(dataset.source_embedding,
                                     hyperparameters=rc)
    model = JointAspectTagger(transformer, rc, 4, 20, dataset.target_names)
    optimizer = get_optimizer(model, rc)
    trainer = Trainer(
                        model,
                        loss,
                        optimizer,
                        rc,
                        dataset,
                        experiment_name,
                        enable_tensorboard=False,
                        verbose=False)
    return trainer

def load_dataset(rc, logger, task):
    dataset = Dataset(
        task,
        logger,
        rc,
        source_index=PREFERENCES.source_index,
        target_vocab_index=PREFERENCES.target_vocab_index,
        data_path=PREFERENCES.data_root,
        train_file=PREFERENCES.data_train,
        valid_file=PREFERENCES.data_validation,
        test_file=PREFERENCES.data_test,
        file_format=PREFERENCES.file_format,
        init_token=None,
        eos_token=None
    )
    dataset.load_data(dsl, verbose=False)
    return dataset

## Preamble - Definitions

In [3]:
PREFERENCES.defaults(
	data_root='./data/data/amazon/splits',
	data_train='train.pkl',    
	data_validation='val.pkl',
	data_test='test.pkl',
	source_index=0,
	target_vocab_index=1,
	file_format='pkl'
)
from data.amazon import amazon_dataset as dsl
main_experiment_name = 'GermEval7_Experiments'
use_cuda = True
STATUS_FAIL = 'fail'
STATUS_OK = 'ok'

 # Definition of experiments
 
 - SpellChecker On
 - Fasttext
 - Single Sentence
 - Combined Sentence
 - Stop Words

In [4]:
baseline = {**default_params, **hyperOpt_goodParams}
print(pprint.pformat(baseline, indent=2))

{ 'att_d_k': 300,
  'att_d_v': 300,
  'batch_size': 12,
  'clip_comments_to': 113,
  'dropout_rate': 0.302424,
  'early_stopping': 5,
  'embedding_dim': 300,
  'embedding_name': '6B',
  'embedding_type': 'fasttext',
  'harmonize_bahn': True,
  'language': 'de',
  'learning_rate_scheduler': { 'noam_learning_rate_factor': 1.418,
                               'noam_learning_rate_warmup': 8000},
  'learning_rate_scheduler_type': <LearningSchedulerType.Noam: 1>,
  'log_every_xth_iteration': -1,
  'model_size': 300,
  'num_encoder_blocks': 2,
  'num_epochs': 25,
  'num_heads': 1,
  'optimizer': { 'adam_beta1': 0.81,
                 'adam_beta2': 0.7173,
                 'adam_eps': 0.000814,
                 'learning_rate': 7.2e-05},
  'optimizer_type': <OptimizerType.Adam: 1>,
  'organic_text_cleaning': False,
  'output_dropout_rate': 0.79602089766246,
  'output_layer': { 'output_conv_kernel_size': 5,
                    'output_conv_num_filters': 300,
                    'output_conv_pa

In [5]:
experiments = [
    {
        'name': 'Baseline',
        'description': 'Baseline. Uses hyperOpt_goodParams without any changes',
        'loss': 1000,
        'f1': -1,
        'rc': {}
    }
]

### Current commit

In [6]:
utils.get_current_git_commit()
print('Current commit: ' + utils.get_current_git_commit())

Current commit: b'e6a4eff'


In [7]:
def objective(rc, experiment):
    run_time = time.time()
    
    # reset loggers
    utils.reset_loggers()
    experiment_name = utils.create_loggers(experiment_name=main_experiment_name)
    logger = logging.getLogger(__name__)
    dataset_logger = logging.getLogger('data_loader')
    
    logger.info('Experiment: ' + experiment['name'])
    logger.info('Description: ' + experiment['description'])
    
    logger.info('Parameters')
    logger.info(rc)
    print('\n\n#########################################################################')
    print('Name: ' + experiment['name'])
    print('Description: ' + experiment['description'])
    print('#########################################################################\n\n')
    print(rc)

    logger.debug('Load dataset')
    try:
        dataset = load_dataset(rc, dataset_logger, rc.task)
    except Exception as err:
        print('Could load dataset: ' + str(err))
        logger.exception("Could not load dataset")
        return {
            'status': STATUS_FAIL,
            'eval_time': time.time() - run_time
        }
    logger.debug('dataset loaded')
    logger.debug('Load model')

    try:
        trainer = load_model(dataset, rc, experiment_name)
    except Exception as err:
        print('Could not load model: ' + str(err))
        logger.exception("Could not load model")
        return {
            'status': STATUS_FAIL,
            'eval_time': time.time() - run_time
        }

    logger.debug('model loaded')

    logger.debug('Begin training')
    model = None
    try:
        result = trainer.train(use_cuda=rc.use_cuda, perform_evaluation=False)
        model = result['model']
    except Exception as err:
        print('Exception while training: ' + str(err))
        logger.exception("Could not complete iteration")
        return {
            'status': STATUS_FAIL,
            'eval_time': time.time() - run_time,
            'best_loss': trainer.get_best_loss(),
            'best_f1': trainer.get_best_f1()
        }

    if math.isnan(trainer.get_best_loss()):
        print('Loss is nan')
        return {
            'status': STATUS_FAIL,
            'eval_time': time.time() - run_time,
            'best_loss': trainer.get_best_loss(),
            'best_f1': trainer.get_best_f1()
        }

    # perform evaluation and log results
    result = None
    try:
        result = trainer.perform_final_evaluation(use_test_set=True, verbose=False)
    except Exception as err:
        logger.exception("Could not complete iteration evaluation.")
        print('Could not complete iteration evaluation: ' + str(err))
        return {
            'status': STATUS_FAIL,
            'eval_time': time.time() - run_time,
            'best_loss': trainer.get_best_loss(),
            'best_f1': trainer.get_best_f1()
        }
    print(f'VAL f1\t{trainer.get_best_f1()} - ({result[1][1]})')
    print(f'VAL loss\t{trainer.get_best_loss()}')
    return {
            'loss': result[1][0],
            'status': STATUS_OK,
            'eval_time': time.time() - run_time,
            'best_loss': trainer.get_best_loss(),
            'best_f1': trainer.get_best_f1(),
            'sample_iterations': trainer.get_num_samples_seen(),
            'iterations': trainer.get_num_iterations(),
            'rc': rc,
            'results': {
                'train': {
                    'loss': result[0][0],
                    'f1': result[0][1]
                },
                'validation': {
                    'loss': result[1][0],
                    'f1': result[1][1]
                },
                'test': {
                    'loss': result[2][0],
                    'f1': result[2][1]
                }
            }
        }

# Perform experiments

In [8]:
for e in experiments:
    name = e['name']
    print(f'#########################################################################\n\nExperiment Name: {name}\n')
    print('#########################################################################\n\n')
    
    # generate rc
    rc = get_default_params(use_cuda=True, overwrite=e['rc'], from_default=baseline)
    result = objective(rc, e)
    
    if result['status'] == STATUS_OK:
        print(f"       .---.\n \
         /     \\\n\
          \\.@-@./\tExperiment: {e['name']}\n\
          /`\\_/`\\\tStatus: {result['status']}\n\
         //  _  \\\\\tLoss: {result['best_loss']}\n\
        | \\     )|_\tf1: {result['best_f1']}\n\
       /`\\_`>  <_/ \\\n\
       \\__/'---'\\__/\n")
    else:
        print(f"       .---.\n \
         /     \\\n\
          \\.@-@./\tExperiment: {e['name']} (FAIL)\n\
          /`\\_/`\\\n\
         //  _  \\\\\\n\
        | \\     )|_\n\
       /`\\_`>  <_/ \\\n\
       \\__/'---'\\__/\n")
    





#########################################################################

Experiment Name: Baseline

#########################################################################


Log path is  C:\Users\felix\OneDrive\Studium\Studium\6. Semester\MA\Project\ABSA-Transformer\logs\GermEval7_Experiments\20190401\0


#########################################################################
Name: Baseline
Description: Baseline. Uses hyperOpt_goodParams without any changes
#########################################################################


+----------------------------------------------------------------------------------+
|                                 Hyperparameters                                  |
+------------------------------+---------------------------------------------------+
|          Parameter           |                       Value                       |
+------------------------------+---------------------------------------------------+
|            kwargs            

A Jupyter Widget

# EP	# IT	tr loss		val loss	f1		acc		duration / total time
1	21k	0.77		0.64		0.314		0.895		4.41m - 4.4m / 0.0m
2	42k	0.60		0.65		0.381		0.909		4.37m - 8.9m / 110.4m
3	63k	0.59		0.59		0.432		0.933		4.27m - 13.2m / 109.4m
4	84k	0.61		0.69		0.527		0.959		4.51m - 17.8m / 107.2m
5	105k	0.61		0.72		0.435		0.933		4.27m - 22.2m / 112.6m
6	126k	0.57		0.64		0.425		0.935		4.28m - 26.5m / 107.7m
7	147k	0.54		0.67		0.362		0.901		4.31m - 30.9m / 108.0m
8	168k	0.53		0.83		0.274		0.839		4.32m - 35.3m / 108.5m
9	189k	0.52		0.65		0.508		0.946		4.32m - 39.7m / 108.7m
VAL f1	0.5270945810837833 - (0.5270945810837833)
VAL loss	0.5930153871484009
       .---.
          /     \
          \.@-@./	Experiment: Baseline
          /`\_/`\	Status: ok
         //  _  \\	Loss: 0.5930153871484009
        | \     )|_	f1: 0.5270945810837833
       /`\_`>  <_/ \
       \__/'---'\__/

